## Oefeningen 2: Transacties

In [ ]:
from database import Database
db = Database("oefeningen.db")

In [ ]:
db.query("SELECT * FROM Transacties ORDER BY RANDOM() LIMIT 5")

### 1. Netto Omzet per maand

In [ ]:
# Bereken maand en totaal bedrag per bestelling
db.query("""
SELECT
    STRFTIME('%m', BestelDatum) AS BestelMaand,
    CASE
        WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
        ELSE PrijsExclusief * Aantal * 1.21
    END AS TotaalInclusief
FROM Transacties
""")

In [ ]:
db.query("""
WITH OmzetMaand AS (
    SELECT
        STRFTIME('%m', BestelDatum) AS BestelMaand,
        CASE
            WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
            ELSE PrijsExclusief * Aantal * 1.21
        END AS TotaalInclusief
    FROM Transacties
)
SELECT
    BestelMaand,
    SUM(TotaalInclusief) AS OmzetNetto
FROM OmzetMaand
GROUP BY BestelMaand
""")

### 2. Cumulatieve omzet per dag

In [ ]:
db.query("""
SELECT
        BestelDatum,
        SUM(
            CASE
                WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
                ELSE PrijsExclusief * Aantal * 1.21
            END
        ) AS DagOmzet
    FROM Transacties
    GROUP BY Besteldatum
""")

In [ ]:
db.query("""
WITH OmzetDag AS (
    SELECT
        BestelDatum,
        SUM(
            CASE
                WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
                ELSE PrijsExclusief * Aantal * 1.21
            END
        ) AS DagOmzet
    FROM Transacties
    GROUP BY BestelDatum
)
SELECT
    BestelDatum,
    DagOmzet,
    SUM(DagOmzet) OVER(ORDER BY BestelDatum) AS OmzetCumulatief
FROM OmzetDag
""")

### 3. Rollend gemiddelde 3 dagen

In [ ]:
db.query("""
WITH OmzetDag AS (
    SELECT
        BestelDatum,
        COUNT(*) AS DagTransacties,
        SUM(Aantal) AS DagArtikelen,
        SUM(
            CASE
                WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
                ELSE PrijsExclusief * Aantal * 1.21
            END
        ) AS DagOmzet
    FROM Transacties
    GROUP BY BestelDatum
)
SELECT
    BestelDatum,
    
    DagTransacties,
    SUM(DagTransacties) OVER(
        ORDER BY BestelDatum
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS Transacties3Dagen,
    
    DagArtikelen,
    SUM(DagArtikelen) OVER(
        ORDER BY BestelDatum
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS Artikelen3Dagen,
    
    DagOmzet,
    SUM(DagOmzet) OVER(
        ORDER BY BestelDatum
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS Omzet3Dagen
FROM OmzetDag
""")

### 4. Verschil met gisteren

In [ ]:
db.query("""
WITH OmzetDag AS (
    SELECT
        BestelDatum,
        SUM(
            CASE
                WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
                ELSE PrijsExclusief * Aantal * 1.21
            END
        ) AS DagOmzet
    FROM Transacties
    GROUP BY BestelDatum
    ORDER BY BestelDatum
)
SELECT
    BestelDatum,
    DagOmzet,
    LAG(DagOmzet) OVER(ORDER BY BestelDatum) AS DagOmzetGisteren,
    DagOmzet - LAG(DagOmzet) OVER(ORDER BY BestelDatum) AS OmzetVerschil
FROM OmzetDag
""").head(15)